# **Orchestrating Machine Learning Workflow with Amazon SageMaker Pipelines**
## Only for Inference job - Using Bring your own code method
### Usecase- Customer Churn Prediction
------------------------------------------------------------------------------------------------------------------------

# Table of Contents

   
* [1. Sagemaker Pipeline](#1.-Sagemaker-Pipeline)
	* [1.1 Architecture](#1.1-Architecture)
	* [1.2 Install predefined Sagemaker libraries](#1.2-Install-predefined-Sagemaker-libraries)
	* [1.3 Import MLOps define functions](#1.3-Import-MLOps-define-functions)
	* [1.4 Define Preprocessing Stage](#1.4-Define-Preprocessing-Stage)
	* [1.5 Define Inference stage](#1.5-Define-Inference-stage)
	* [1.6 Define Postprocess steps](#1.6-Define-Postprocess-steps)
	* [1.7 Define required parameters for get pipeline](#1.7-Define-required-parameters-for-get-pipeline)   


# 1. Sagemaker Pipeline

SageMaker Pipelines supports the following activities, which are demonstrated in this notebook:

- Pipelines - A DAG of steps and conditions to orchestrate SageMaker jobs and resource creation.
- Processing job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
- Training job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
- Register model steps - A step that creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
- Create model steps - A step that creates a model for use in transform steps or later publication as an endpoint.
- Transform job steps - A batch transform to preprocess datasets to remove noise or bias that interferes with training or inference from a dataset, get inferences from large datasets, and run inference when a persistent endpoint is not needed.
- Post processing - (Optional) A step that filtering the final predicted output base : In here we don't include that step into our pipeline
- Parametrized Pipeline executions - Enables variation in pipeline executions according to specified parameters.

### 1.1 Architecture

This **inference** pipeline contains preprocess, inference, create model and post-process steps.

![architecture](images/pic9.PNG)

Lets beginning the code

![workflow](images/pic10.PNG)

## 1.2Install predefined Sagemaker libraries


Initailly we have to install AWS predefined Sagemaker libraries.

In [231]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:

import os
import pytz
from datetime import datetime

import boto3
import sagemaker
import sagemaker.session

from sagemaker.transformer import Transformer
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput,TransformInput,CreateModelInput
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)
from sagemaker.network import NetworkConfig
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.conditions import (
    ConditionGreaterThanOrEqualTo,
)
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import (
    ProcessingStep,
    TrainingStep,
    TransformStep,
    CreateModelStep
)
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.model import Model


sess = sagemaker.Session()

![workflow](images/pic11.PNG)

## 1.3 Import MLOps define functions

Next step is start session and in this process we are defining our <b> AWS region, sagemaker client, boto 3 session</b> and <b>default s3 bucket </b>.

In [232]:
def get_session(region, default_bucket):
    """Gets the sagemaker session based on the region.
    Args:
        region: the aws region to start the session
        default_bucket: the bucket to use for storing the artifacts
    Returns:
        `sagemaker.session.Session instance
    """

    boto_session = boto3.Session(region_name=region)

    sagemaker_client = boto_session.client("sagemaker")
    runtime_client = boto_session.client("sagemaker-runtime")
    return sagemaker.session.Session(
        boto_session=boto_session,
        sagemaker_client=sagemaker_client,
        sagemaker_runtime_client=runtime_client,
        default_bucket=default_bucket,
    )

Enter the project name for import config files

In [233]:
project_name = "awsworkshop"
job_name='inference'
user_name = "mlops"

In [240]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

env = 'dev'

import json
import boto3

def getJsonData(bucket_name,key_name):
    print("[LOG]", bucket_name,'---------')
    print("[LOG]", key_name,'--------------')
      
    s3_client = boto3.client('s3')
    csv_obj = s3_client.get_object(Bucket=bucket_name, Key=key_name)
    
    body = csv_obj['Body']
    
    json_string = body.read().decode('utf-8')
    json_content = json.loads(json_string)
    
    return json_content

config_bucket = ParameterString(
        name="ConfigBucket",
        default_value=f"dlk-cloud-tier-8-code-ml-{env}",  # Change this to point to the s3 location of your raw input data.
    )
config_s3_prefix = ParameterString(
        name="ConfigS3Prefix",
        default_value=f'config_files/pipeline_config/{project_name}/{job_name}/{user_name}/pipeline_config.json',  # Change this to point to the s3 location of your raw input data.
    )    
    
config = getJsonData(config_bucket,config_s3_prefix)

[LOG] dlk-cloud-tier-8-code-ml-dev ---------
[LOG] config_files/pipeline_config/awsworkshop/inference/mlops/pipeline_config.json --------------


In [235]:
### calling the leatest model
def model_with_pipeline():

    env='dev'
    s3_path = f"s3://dlk-cloud-tier-9-training-ml-{env}/"
    s3 = boto3.client('s3')
    all_objects = s3.list_objects(Bucket = f"dlk-cloud-tier-9-training-ml-{env}", Prefix=f"awsworkshop/")
    model_folder=all_objects['Contents'][-1]['Key']
    model_path=s3_path+model_folder
    
    #model_path = "s3://dlk-cloud-tier-9-training-ml-dev/awsworkshop/2022-02-28/pipelines-jx3s70a5peup-awsworkshop-team1-tr-itk6CKL57c/output/model.tar.gz"
    return model_path

![workflow](images/pic12.PNG)

## 1.4 Define Preprocessing Stage

This is the script used in preprocessing 

In [227]:
!pygmentize awsworkshop_team1_inf_preprocessing/preprocessing.py

import boto3
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
import argparse
import os
import warnings
warnings.simplefilter(action='ignore')
import json

print("import your necessary libraries in here") 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


print("enter your project name and enviornment here")
project_name = "aws_workshop"
env = 'dev'

print("loading the stage config data from s3")

def getJsonData(bucket_name,key_name):
    print("[LOG]", bucket_name,'---------')
    print("[LOG]", key_name,'--------------')
      
    s3_client = boto3.client('s3')
    csv_obj = s3_client.get_object(Bucket=bucket_name, Key=key_name)
    
    body = csv_obj['Body']
    
    json_string = body.read().decode('utf-8')
    json_content = json.loads(json_string)
    
    return json_content

print("setting the param

This is the code for run the preprocessing part. You can add more inputs or outputs according to your requirement. Please refer the guideline document for more details.

In [51]:
def get_pipeline(
    region,
    subnets,
    security_group_ids,
    role=None,
    default_bucket=None,
    model_package_group_name=config['pipeline']['pipeline_config']['model_package_group_name'],  # Choose any name
    pipeline_name=config['pipeline']['pipeline_config']['pipeline_name'],  # You can find your pipeline name in the Studio UI (project -> Pipelines -> name)
    base_job_prefix=config['pipeline']['pipeline_config']['base_job_prefix'],  # Choose any name
    env=config['pipeline']['pipeline_config']['env']
):
    """Gets a SageMaker ML Pipeline instance working with on CustomerChurn data.
    Args:
        region: AWS region to create and run the pipeline.
        role: IAM role to create and run steps and pipeline.
        default_bucket: the bucket to use for storing the artifacts
    Returns:
        an instance of a pipeline
    """
    #data versioning control using date
    srilanka_tz = pytz.timezone('Asia/Colombo')
    s3 = boto3.client('s3')
    date_folder = datetime.now(srilanka_tz).strftime("%Y-%m-%d")
    
    #working with input data path
    input_data = config['pipeline']['inputs']['input1']
    #input_data2 = config['pipeline']['inputs']['input2']
    #input_data3 = config['pipeline']['inputs']['input3']
    #input_data4 = config['pipeline']['inputs']['input4']
    #input_data5 = config['pipeline']['inputs']['input5']
    
    #working with output data path   
    preprocessed_output1 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output1/"
    #preprocessed_output2 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output2/"
    #preprocessed_output3 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output3/"
    #preprocessed_output4 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output4/"
    #preprocessed_output5 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output5/"
    
    generic_tags=[{'Key': 'EnvironmentName', 'Value': env}, 
                  {'Key': 'ProjectName', 'Value': config['pipeline']['generic_tags']['ProjectName']},
                  {'Key': 'DepartmentName', 'Value': config['pipeline']['generic_tags']['DepartmentName']}, 
                  {'Key': 'UsecaseName', 'Value': config['pipeline']['generic_tags']['UsecaseName']},
                  {'Key': 'ResourceName', 'Value': config['pipeline']['generic_tags']['ResourceName']}, 
                  {'Key': 'OwnerName', 'Value': config['pipeline']['generic_tags']['OwnerName']},
                  {'Key': 'BUName', 'Value': config['pipeline']['generic_tags']['BUName']} ]
    
    
    sagemaker_session = get_session(region, default_bucket)
    if role is None:
        role = sagemaker.session.get_execution_role(sagemaker_session)
    account_id = boto3.client("sts").get_caller_identity().get("Account")
    region = boto3.session.Session().region_name

    # Parameters for pipeline execution

    ####### --------------------- PREPROCESSING --------------------------------------------------------------------

    ecr_repository = config['preprocessing']['estimator_config']['ecr_repository']
    tag = ":latest"
    uri_suffix = "amazonaws.com"
    
    preprocessing_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
        account_id, region, uri_suffix, ecr_repository + tag
    )
        
    script_processor = ScriptProcessor(
         command = ["python3"],
         image_uri = preprocessing_repository_uri,
         role = role,
         instance_count = config['preprocessing']['estimator_config']['instance_count'],
         instance_type = config['preprocessing']['estimator_config']['instance_type'],
         tags = generic_tags + [{'Key': 'JobType', 'Value': 'Preprocessing'}],
         network_config = NetworkConfig(subnets=subnets.split(':'), security_group_ids=security_group_ids.split(':'))
    )
    
    step_preprocess = ProcessingStep(
        name= f"{base_job_prefix}-preprocessing",
        processor= script_processor, 
        code= config['preprocessing']['inputs']['code'],
        inputs= [ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
                #ProcessingInput(source=input_data2, destination="/opt/ml/processing/input2"),
                #ProcessingInput(source=input_data3, destination="/opt/ml/processing/input3"),
                #ProcessingInput(source=input_data4, destination="/opt/ml/processing/input4"),
                #ProcessingInput(source=input_data5, destination="/opt/ml/processing/input5"),
               ],
        outputs= [
            ProcessingOutput(output_name="output1", destination=preprocessed_output1, source="/opt/ml/processing/output1"),
            #ProcessingOutput(output_name="output2", destination=preprocessed_output2,  source="/opt/ml/processing/output2"),
            #ProcessingOutput(output_name="output3", destination=preprocessed_output3,  source="/opt/ml/processing/output3"),
            #ProcessingOutput(output_name="output4", destination=preprocessed_output2,  source="/opt/ml/processing/output4"),
            #ProcessingOutput(output_name="output5", destination=preprocessed_output2,  source="/opt/ml/processing/output5"),
        ]
    )
    
    # ========================================= PIPELINE ORCHESTRATION ================================================
    
    # Pipeline instance
    pipeline = Pipeline(
        name=pipeline_name+env,
        parameters=[
        ],
        steps=[
            step_preprocess,
              ],
        sagemaker_session=sagemaker_session,
    )
    return pipeline


![workflow](images/pic13.PNG)

## 1.5 Define Inference stage

We used this script for inference stage.

In [228]:
!pygmentize awsworkshop_team1_inf_inferencing/model/predictor.py

# This is the file that implements a flask server to do inferences. It's the file that you will modify to
# implement the scoring for your own algorithm.

from __future__ import print_function

import os
import json
import pickle
import io
import sys
import signal
import traceback

import flask
import boto3
import pandas as pd
import numpy as np
#import statsmodels.api as sm

#Temporarly remove this path and giving the hardcode path
prefix = '/opt/ml/'
model_path = os.path.join(prefix, 'model')
s3_client = boto3.client('s3')


# A singleton for holding the model. This simply loads the model and holds it.
# It has a predict function that does a prediction based on the model and the input data.

class CustomerChurn(object):
    model = None  # Where we keep the model when it's loaded
    complete_model = None
    model_flag = False

    # @classmethod
    # def get_model(cls):
    #     """Get the model object for this instance, loading it if it's not already loaded."""
    #     if cls.

Please execute bellow code to create the pipeline. This pipeline contains  **preprocessing and inference** steps together

In [ ]:
def get_pipeline(
    region,
    subnets,
    security_group_ids,
    role=None,
    default_bucket=None,
    model_package_group_name=config['pipeline']['pipeline_config']['model_package_group_name'],  # Choose any name
    pipeline_name=config['pipeline']['pipeline_config']['pipeline_name'],  # You can find your pipeline name in the Studio UI (project -> Pipelines -> name)
    base_job_prefix=config['pipeline']['pipeline_config']['base_job_prefix'],  # Choose any name
    env=config['pipeline']['pipeline_config']['env']
):
    """Gets a SageMaker ML Pipeline instance working with on CustomerChurn data.
    Args:
        region: AWS region to create and run the pipeline.
        role: IAM role to create and run steps and pipeline.
        default_bucket: the bucket to use for storing the artifacts
    Returns:
        an instance of a pipeline
    """
    
    
    #data versioning control using date
    srilanka_tz = pytz.timezone('Asia/Colombo')
    s3 = boto3.client('s3')
    date_folder = datetime.now(srilanka_tz).strftime("%Y-%m-%d")
    
    config = getJsonData(config_bucket,config_s3_prefix)
    #working with input data path
    input_data = config['pipeline']['inputs']['input1']
    #input_data2 = config['pipeline']['inputs']['input2']
    #input_data3 = config['pipeline']['inputs']['input3']
    #input_data4 = config['pipeline']['inputs']['input4']
    #input_data5 = config['pipeline']['inputs']['input5']
    
    #working with output data path   
    preprocessed_output1 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output1/"
    #preprocessed_output2 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output2/"
    #preprocessed_output3 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output3/"
    #preprocessed_output4 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output4/"
    #preprocessed_output5 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output5/"
    
    postprocessed_output1= f"s3://dlk-cloud-tier-12-postprocessed-ml-{env}/{project_name}/postprocessing/{date_folder}/output1/"
    
    generic_tags=[{'Key': 'EnvironmentName', 'Value': env}, 
                  {'Key': 'ProjectName', 'Value': config['pipeline']['generic_tags']['ProjectName']},
                  {'Key': 'DepartmentName', 'Value': config['pipeline']['generic_tags']['DepartmentName']}, 
                  {'Key': 'UsecaseName', 'Value': config['pipeline']['generic_tags']['UsecaseName']},
                  {'Key': 'ResourceName', 'Value': config['pipeline']['generic_tags']['ResourceName']}, 
                  {'Key': 'OwnerName', 'Value': config['pipeline']['generic_tags']['OwnerName']},
                  {'Key': 'BUName', 'Value': config['pipeline']['generic_tags']['BUName']} ]
    
    
    sagemaker_session = get_session(region, default_bucket)
    if role is None:
        role = sagemaker.session.get_execution_role(sagemaker_session)
    account_id = boto3.client("sts").get_caller_identity().get("Account")
    region = boto3.session.Session().region_name

    # Parameters for pipeline execution
    
    

    ####### --------------------- PREPROCESSING --------------------------------------------------------------------

    ecr_repository = config['preprocessing']['estimator_config']['ecr_repository']
    tag = ":latest"
    uri_suffix = "amazonaws.com"
    
    preprocessing_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
        account_id, region, uri_suffix, ecr_repository + tag
    )
        
    script_processor = ScriptProcessor(
         command = ["python3"],
         image_uri = preprocessing_repository_uri,
         role = role,
         instance_count = config['preprocessing']['estimator_config']['instance_count'],
         instance_type = config['preprocessing']['estimator_config']['instance_type'],
         tags = generic_tags + [{'Key': 'JobType', 'Value': 'Preprocessing'}],
         network_config = NetworkConfig(subnets=subnets.split(':'), security_group_ids=security_group_ids.split(':'))
    )
    
    step_preprocess = ProcessingStep(
        name= f"{base_job_prefix}-preprocessing",
        processor= script_processor, 
        code= config['preprocessing']['inputs']['code'],
        inputs= [ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
                #ProcessingInput(source=input_data2, destination="/opt/ml/processing/input2"),
                #ProcessingInput(source=input_data3, destination="/opt/ml/processing/input3"),
                #ProcessingInput(source=input_data4, destination="/opt/ml/processing/input4"),
                #ProcessingInput(source=input_data5, destination="/opt/ml/processing/input5"),
               ],
        outputs= [
            ProcessingOutput(output_name="output1", destination=preprocessed_output1, source="/opt/ml/processing/output1"),
            #ProcessingOutput(output_name="output2", destination=preprocessed_output2,  source="/opt/ml/processing/output2"),
            #ProcessingOutput(output_name="output3", destination=preprocessed_output3,  source="/opt/ml/processing/output3"),
            #ProcessingOutput(output_name="output4", destination=preprocessed_output2,  source="/opt/ml/processing/output4"),
            #ProcessingOutput(output_name="output5", destination=preprocessed_output2,  source="/opt/ml/processing/output5"),
        ]
    )
    
    
    ############## Inference Job ##############################################
    config = getJsonData(config_bucket,config_s3_prefix)
    
    ecr_repository = config['inference']['estimator_config']['ecr_repository']
    tag = ":latest"
    uri_suffix = "amazonaws.com"
    
    create_model_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
        account_id, region, uri_suffix, ecr_repository + tag
    )
    
    image = create_model_repository_uri
    #est_config = config["train_model"]["estimator_config"]
    #generic_tags=[{'Key': 'EnvironmentName', 'Value': env}, {'Key': 'ProjectName', 'Value': 'dialog_migration'},{'Key': 'DepartmentName', 'Value': 'dialog_ml'}, {'Key': 'BUName', 'Value': 'Network'},  {'Key': 'UsecaseName', 'Value': 'anomaly-detection'}, {'Key': 'ResourceName', 'Value': 'sagemaker_jobs'}, {'Key': 'OwnerName', 'Value': 'utkarsh_blazeclan'} ]

    #Batch Transform job
    
    #model_name = step_create_model.properties.ModelName
    job_name = config["create_model"]["estimator_config"]["job_name"]
    instance_type = config["create_model"]["estimator_config"]["instance_type"]
    step_create_model = CreateModelStep(
        name=f"{base_job_prefix}-createmodel",
        model=Model(image, 
                    model_data=model_with_pipeline(),
                    role = role,
                    sagemaker_session = sagemaker_session,
                    name = job_name,
                    vpc_config = {'Subnets':subnets.split(':'),
                                    'SecurityGroupIds': security_group_ids.split(':')}),
        inputs=CreateModelInput(instance_type=instance_type,
                               #accelerator_type="ml.eia1.medium"
                               )
    )
    
    model_name = step_create_model.properties.ModelName
    instance_count = config["inference"]["estimator_config"]["instance_count"]
    instance_type = config["inference"]["estimator_config"]["instance_type"]
    content_type = config["inference"]["estimator_config"]["content_type"]
    
    transformer = Transformer(model_name=model_name,
                              instance_count=instance_count,
                              strategy='SingleRecord',
                              #max_payload=15, 
                              assemble_with="Line",
                              instance_type=instance_type,
                              output_path=f"s3://dlk-cloud-tier-11-inference-ml-{env}/{project_name}/batch/{date_folder}/",
                              tags = generic_tags + [{'Key': 'JobName', 'Value': 'Inference'}]
                )
    
    step_transform = TransformStep(
        name=f"{base_job_prefix}-batchtransform",
        transformer=transformer,
        inputs=TransformInput(data = step_preprocess.properties.ProcessingOutputConfig.Outputs["output1"].S3Output.S3Uri,
            ##data="s3://dlk-cloud-tier-10-preprocessed-ml-dev/awsworkshop/inference/2022-02-28/output1/data_output.csv.gz",
                             split_type="Line",
                               compression_type = 'Gzip', 
                              content_type=content_type)
    )
    

    # ========================================= PIPELINE ORCHESTRATION ================================================
    
    # Pipeline instance
    pipeline = Pipeline(
        name=pipeline_name+env,
        parameters=[
            #model_path,
            #model_approval_status
        ],
        steps=[
            step_preprocess,
            step_create_model,
            step_transform,
              ],
        sagemaker_session=sagemaker_session,
    )
    return pipeline


![workflow](images/pic14.PNG)

## 1.6 Define Postprocess steps 

Please execute bellow code to create the pipeline.This pipeline contains **preprocessing,inference and postprocess** steps together

In [241]:
def get_pipeline(
    region,
    subnets,
    security_group_ids,
    role=None,
    default_bucket=None,
    model_package_group_name=config['pipeline']['pipeline_config']['model_package_group_name'],  # Choose any name
    pipeline_name=config['pipeline']['pipeline_config']['pipeline_name'],  # You can find your pipeline name in the Studio UI (project -> Pipelines -> name)
    base_job_prefix=config['pipeline']['pipeline_config']['base_job_prefix'],  # Choose any name
    env=config['pipeline']['pipeline_config']['env']
):
    """Gets a SageMaker ML Pipeline instance working with on CustomerChurn data.
    Args:
        region: AWS region to create and run the pipeline.
        role: IAM role to create and run steps and pipeline.
        default_bucket: the bucket to use for storing the artifacts
    Returns:
        an instance of a pipeline
    """
    
    
    #data versioning control using date
    srilanka_tz = pytz.timezone('Asia/Colombo')
    s3 = boto3.client('s3')
    date_folder = datetime.now(srilanka_tz).strftime("%Y-%m-%d")
    
    config = getJsonData(config_bucket,config_s3_prefix)
    #working with input data path
    input_data = config['pipeline']['inputs']['input1']
    #input_data2 = config['pipeline']['inputs']['input2']
    #input_data3 = config['pipeline']['inputs']['input3']
    #input_data4 = config['pipeline']['inputs']['input4']
    #input_data5 = config['pipeline']['inputs']['input5']
    
    #working with output data path   
    preprocessed_output1 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output1/"
    #preprocessed_output2 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output2/"
    #preprocessed_output3 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output3/"
    #preprocessed_output4 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output4/"
    #preprocessed_output5 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output5/"
    
    postprocessed_output1= f"s3://dlk-cloud-tier-12-postprocessed-ml-{env}/{project_name}/postprocessing/{date_folder}/output1/"
    
    generic_tags=[{'Key': 'EnvironmentName', 'Value': env}, 
                  {'Key': 'ProjectName', 'Value': config['pipeline']['generic_tags']['ProjectName']},
                  {'Key': 'DepartmentName', 'Value': config['pipeline']['generic_tags']['DepartmentName']}, 
                  {'Key': 'UsecaseName', 'Value': config['pipeline']['generic_tags']['UsecaseName']},
                  {'Key': 'ResourceName', 'Value': config['pipeline']['generic_tags']['ResourceName']}, 
                  {'Key': 'OwnerName', 'Value': config['pipeline']['generic_tags']['OwnerName']},
                  {'Key': 'BUName', 'Value': config['pipeline']['generic_tags']['BUName']} ]
    
    
    sagemaker_session = get_session(region, default_bucket)
    if role is None:
        role = sagemaker.session.get_execution_role(sagemaker_session)
    account_id = boto3.client("sts").get_caller_identity().get("Account")
    region = boto3.session.Session().region_name

    # Parameters for pipeline execution
    
    

    ####### --------------------- PREPROCESSING --------------------------------------------------------------------

    ecr_repository = config['preprocessing']['estimator_config']['ecr_repository']
    tag = ":latest"
    uri_suffix = "amazonaws.com"
    
    preprocessing_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
        account_id, region, uri_suffix, ecr_repository + tag
    )
        
    script_processor = ScriptProcessor(
         command = ["python3"],
         image_uri = preprocessing_repository_uri,
         role = role,
         instance_count = config['preprocessing']['estimator_config']['instance_count'],
         instance_type = config['preprocessing']['estimator_config']['instance_type'],
         tags = generic_tags + [{'Key': 'JobType', 'Value': 'Preprocessing'}],
         network_config = NetworkConfig(subnets=subnets.split(':'), security_group_ids=security_group_ids.split(':'))
    )
    
    step_preprocess = ProcessingStep(
        name= f"{base_job_prefix}-preprocessing",
        processor= script_processor, 
        code= config['preprocessing']['inputs']['code'],
        inputs= [ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),
                #ProcessingInput(source=input_data2, destination="/opt/ml/processing/input2"),
                #ProcessingInput(source=input_data3, destination="/opt/ml/processing/input3"),
                #ProcessingInput(source=input_data4, destination="/opt/ml/processing/input4"),
                #ProcessingInput(source=input_data5, destination="/opt/ml/processing/input5"),
               ],
        outputs= [
            ProcessingOutput(output_name="output1", destination=preprocessed_output1, source="/opt/ml/processing/output1"),
            #ProcessingOutput(output_name="output2", destination=preprocessed_output2,  source="/opt/ml/processing/output2"),
            #ProcessingOutput(output_name="output3", destination=preprocessed_output3,  source="/opt/ml/processing/output3"),
            #ProcessingOutput(output_name="output4", destination=preprocessed_output2,  source="/opt/ml/processing/output4"),
            #ProcessingOutput(output_name="output5", destination=preprocessed_output2,  source="/opt/ml/processing/output5"),
        ]
    )
    
    
    ############## Inference Job ##############################################
    config = getJsonData(config_bucket,config_s3_prefix)
    
    ecr_repository = config['inference']['estimator_config']['ecr_repository']
    tag = ":latest"
    uri_suffix = "amazonaws.com"
    
    create_model_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
        account_id, region, uri_suffix, ecr_repository + tag
    )
    
    image = create_model_repository_uri
    #est_config = config["train_model"]["estimator_config"]
    #generic_tags=[{'Key': 'EnvironmentName', 'Value': env}, {'Key': 'ProjectName', 'Value': 'dialog_migration'},{'Key': 'DepartmentName', 'Value': 'dialog_ml'}, {'Key': 'BUName', 'Value': 'Network'},  {'Key': 'UsecaseName', 'Value': 'anomaly-detection'}, {'Key': 'ResourceName', 'Value': 'sagemaker_jobs'}, {'Key': 'OwnerName', 'Value': 'utkarsh_blazeclan'} ]

    #Batch Transform job
    
    #model_name = step_create_model.properties.ModelName
    job_name = config["create_model"]["estimator_config"]["job_name"]
    instance_type = config["create_model"]["estimator_config"]["instance_type"]
    step_create_model = CreateModelStep(
        name=f"{base_job_prefix}-createmodel",
        model=Model(image, 
                    model_data=model_with_pipeline(),
                    role = role,
                    sagemaker_session = sagemaker_session,
                    name = job_name,
                    vpc_config = {'Subnets':subnets.split(':'),
                                    'SecurityGroupIds': security_group_ids.split(':')}),
        inputs=CreateModelInput(instance_type=instance_type,
                               #accelerator_type="ml.eia1.medium"
                               )
    )
    
    model_name = step_create_model.properties.ModelName
    instance_count = config["inference"]["estimator_config"]["instance_count"]
    instance_type = config["inference"]["estimator_config"]["instance_type"]
    content_type = config["inference"]["estimator_config"]["content_type"]
    
    transformer = Transformer(model_name=model_name,
                              instance_count=instance_count,
                              strategy='SingleRecord',
                              #max_payload=15, 
                              assemble_with="Line",
                              instance_type=instance_type,
                              output_path=f"s3://dlk-cloud-tier-11-inference-ml-{env}/{project_name}/batch/{date_folder}/",
                              tags = generic_tags + [{'Key': 'JobName', 'Value': 'Inference'}]
                )
    
    step_transform = TransformStep(
        name=f"{base_job_prefix}-batchtransform",
        transformer=transformer,
        inputs=TransformInput(data = step_preprocess.properties.ProcessingOutputConfig.Outputs["output1"].S3Output.S3Uri,
            ##data="s3://dlk-cloud-tier-10-preprocessed-ml-dev/awsworkshop/inference/2022-02-28/output1/data_output.csv.gz",
                             split_type="Line",
                               compression_type = 'Gzip', 
                              content_type=content_type)
    )
    
    ####### --------------------- Post-PREPROCESSING --------------------------------------------------------------------

    ecr_repository = config['preprocessing']['estimator_config']['ecr_repository']
    tag = ":latest"
    uri_suffix = "amazonaws.com"
    
    preprocessing_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
        account_id, region, uri_suffix, ecr_repository + tag
    )
        
    script_postprocessor = ScriptProcessor(
         command = ["python3"],
         image_uri = preprocessing_repository_uri,
         role = role,
         instance_count = config['preprocessing']['estimator_config']['instance_count'],
         instance_type = config['preprocessing']['estimator_config']['instance_type'],
         tags = generic_tags + [{'Key': 'JobType', 'Value': 'Preprocessing'}],
         network_config = NetworkConfig(subnets=subnets.split(':'), security_group_ids=security_group_ids.split(':'))
    )
    
    step_postpreprocess = ProcessingStep(
        name= f"{base_job_prefix}-postpreprocessing",
        processor= script_postprocessor, 
        code= config["postpreprocessing"]['inputs']['code'],
        inputs= [ProcessingInput(source=step_transform.properties.TransformOutput.S3OutputPath, destination="/opt/ml/processing/input",s3_data_type='S3Prefix'),
                ProcessingInput(source=step_preprocess.properties.ProcessingOutputConfig.Outputs["output1"].S3Output.S3Uri, destination="/opt/ml/processing/input1"),

               ],
        outputs= [
            ProcessingOutput(output_name="post_output1", destination=postprocessed_output1, source="/opt/ml/processing/post_output1"),
            #ProcessingOutput(output_name="post_output2", destination=postprocessed_output2,  source="/opt/ml/processing/post_output2"),
            #ProcessingOutput(output_name="post_output3", destination=postprocessed_output3,  source="/opt/ml/processing/post_output3"),
            #ProcessingOutput(output_name="post_output4", destination=postprocessed_output4,  source="/opt/ml/processing/post_output4"),
            #ProcessingOutput(output_name="post_output5", destination=postprocessed_output5,  source="/opt/ml/processing/post_output5"),
        ]
    )    
    # ========================================= PIPELINE ORCHESTRATION ================================================
    
    # Pipeline instance
    pipeline = Pipeline(
        name=pipeline_name+env,
        parameters=[
            #model_path,
            #model_approval_status
        ],
        steps=[
            step_preprocess,
            step_create_model,
            step_transform,
            step_postpreprocess
              ],
        sagemaker_session=sagemaker_session,
    )
    return pipeline


![workflow](images/pic15.PNG)

This is the script used for postprocessing 

In [229]:
!pygmentize awsworkshop_team1_inf_postprocessing/postprocessing.py

#import only the necessary libraries

import glob 
import pyarrow.parquet as pq
import pickle
import datetime
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
import argparse
import os
import boto3
import json

print("import your necessary libraries in here") 
import pandas as pd 
import numpy as np


print('successflly import the libaries')

print("enter your project name and enviornment here")
project_name = "aws_workshop"
env = 'dev'

print("loading the stage config data from s3")

def getJsonData(bucket_name,key_name):
    print("[LOG]", bucket_name,'---------')
    print("[LOG]", key_name,'--------------')
      
    s3_client = boto3.client('s3')
    csv_obj = s3_client.get_object(Bucket=bucket_name, Key=key_name)
    
    body = csv_obj['Body']
    
    json_string = body.read().decode('utf-8')
    json_content = json.loads(json_string)
    
    return json_content

print("setting the paramet

Please execute bellow code to create the pipeline.This pipeline contains **postpreprocessing** steps.

In [203]:
def get_pipeline(
    region,
    subnets,
    security_group_ids,
    role=None,
    default_bucket=None,
    model_package_group_name=config['pipeline']['pipeline_config']['model_package_group_name'],  # Choose any name
    pipeline_name=config['pipeline']['pipeline_config']['pipeline_name'],  # You can find your pipeline name in the Studio UI (project -> Pipelines -> name)
    base_job_prefix=config['pipeline']['pipeline_config']['base_job_prefix'],  # Choose any name
    env=config['pipeline']['pipeline_config']['env']
):
    """Gets a SageMaker ML Pipeline instance working with on CustomerChurn data.
    Args:
        region: AWS region to create and run the pipeline.
        role: IAM role to create and run steps and pipeline.
        default_bucket: the bucket to use for storing the artifacts
    Returns:
        an instance of a pipeline
    """
    
    
    #data versioning control using date
    srilanka_tz = pytz.timezone('Asia/Colombo')
    s3 = boto3.client('s3')
    date_folder = datetime.now(srilanka_tz).strftime("%Y-%m-%d")
    
    config = getJsonData(config_bucket,config_s3_prefix)
    #working with input data path
    input_data = config['pipeline']['inputs']['input1']
    #input_data2 = config['pipeline']['inputs']['input2']
    #input_data3 = config['pipeline']['inputs']['input3']
    #input_data4 = config['pipeline']['inputs']['input4']
    #input_data5 = config['pipeline']['inputs']['input5']
    
    #working with output data path   
    preprocessed_output1 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output1/"
    #preprocessed_output2 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output2/"
    #preprocessed_output3 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output3/"
    #preprocessed_output4 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output4/"
    #preprocessed_output5 = f"s3://dlk-cloud-tier-10-preprocessed-ml-{env}/{project_name}/inference/{date_folder}/output5/"
    
    
    postprocessed_output1= f"s3://dlk-cloud-tier-12-postprocessed-ml-{env}/{project_name}/postprocessing/{date_folder}/output1/"
    
    
    generic_tags=[{'Key': 'EnvironmentName', 'Value': env}, 
                  {'Key': 'ProjectName', 'Value': config['pipeline']['generic_tags']['ProjectName']},
                  {'Key': 'DepartmentName', 'Value': config['pipeline']['generic_tags']['DepartmentName']}, 
                  {'Key': 'UsecaseName', 'Value': config['pipeline']['generic_tags']['UsecaseName']},
                  {'Key': 'ResourceName', 'Value': config['pipeline']['generic_tags']['ResourceName']}, 
                  {'Key': 'OwnerName', 'Value': config['pipeline']['generic_tags']['OwnerName']},
                  {'Key': 'BUName', 'Value': config['pipeline']['generic_tags']['BUName']} ]
    
    
    sagemaker_session = get_session(region, default_bucket)
    if role is None:
        role = sagemaker.session.get_execution_role(sagemaker_session)
    account_id = boto3.client("sts").get_caller_identity().get("Account")
    region = boto3.session.Session().region_name

    # Parameters for pipeline execution
    
    

    ####### --------------------- Post-PREPROCESSING --------------------------------------------------------------------

    ecr_repository = config['preprocessing']['estimator_config']['ecr_repository']
    tag = ":latest"
    uri_suffix = "amazonaws.com"
    
    preprocessing_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
        account_id, region, uri_suffix, ecr_repository + tag
    )
        
    script_postprocessor = ScriptProcessor(
         command = ["python3"],
         image_uri = preprocessing_repository_uri,
         role = role,
         instance_count = config['preprocessing']['estimator_config']['instance_count'],
         instance_type = config['preprocessing']['estimator_config']['instance_type'],
         tags = generic_tags + [{'Key': 'JobType', 'Value': 'Preprocessing'}],
         network_config = NetworkConfig(subnets=subnets.split(':'), security_group_ids=security_group_ids.split(':'))
    )
    
    step_postpreprocess = ProcessingStep(
        name= f"{base_job_prefix}-postpreprocessing",
        processor= script_postprocessor, 
        code= config["postpreprocessing"]['inputs']['code'],
        inputs= [ProcessingInput(source='s3://dlk-cloud-tier-11-inference-ml-dev/awsworkshop/batch/2022-03-01/data_output.csv.gz.out', destination="/opt/ml/processing/input"),
                ProcessingInput(source='s3://dlk-cloud-tier-10-preprocessed-ml-dev/awsworkshop/inference/2022-03-01/output1/data_output.csv.gz', destination="/opt/ml/processing/input1"),
                #ProcessingInput(source=input_data3, destination="/opt/ml/processing/input3"),
                #ProcessingInput(source=input_data4, destination="/opt/ml/processing/input4"),
                #ProcessingInput(source=input_data5, destination="/opt/ml/processing/input5"),
               ],
        outputs= [
            ProcessingOutput(output_name="output1", destination=postprocessed_output1, source="/opt/ml/processing/output1"),
            #ProcessingOutput(output_name="output2", destination=preprocessed_output2,  source="/opt/ml/processing/output2"),
            #ProcessingOutput(output_name="output3", destination=preprocessed_output3,  source="/opt/ml/processing/output3"),
            #ProcessingOutput(output_name="output4", destination=preprocessed_output2,  source="/opt/ml/processing/output4"),
            #ProcessingOutput(output_name="output5", destination=preprocessed_output2,  source="/opt/ml/processing/output5"),
        ]
    )
    

    # ========================================= PIPELINE ORCHESTRATION ================================================
    
    # Pipeline instance
    pipeline = Pipeline(
        name=pipeline_name+env,
        parameters=[
            #model_path,
            #model_approval_status
        ],
        steps=[
            step_postpreprocess,
            
              ],
        sagemaker_session=sagemaker_session,
    )
    return pipeline


![workflow](images/pic16.PNG)

## 1.7 Define required parameters for get pipeline

Define subnets and parameters for get_pipeline function here

In [242]:
dev_subnets = 'subnet-036d6b39301b4a41a'
dev_sg = 'sg-0970930479bbef573'

In [243]:
region ='ap-southeast-1'
role='arn:aws:iam::120582440665:role/Sagemaker'
#role=None
default_bucket='pipeline-sagemaker-test'
pipeline_def = get_pipeline(region, 
                            dev_subnets, 
                            dev_sg, 
                            role,
                            default_bucket, 
                            env='dev')

[LOG] dlk-cloud-tier-8-code-ml-dev ---------
[LOG] config_files/pipeline_config/awsworkshop/inference/mlops/pipeline_config.json --------------
[LOG] dlk-cloud-tier-8-code-ml-dev ---------
[LOG] config_files/pipeline_config/awsworkshop/inference/mlops/pipeline_config.json --------------


![workflow](images/pic17.PNG)

In [244]:
pipeline_def.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-southeast-1:120582440665:pipeline/awsworkshop-team2-inference-pipeline-dev',
 'ResponseMetadata': {'RequestId': '816043c3-ab94-4207-a665-2709f2bf7e52',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '816043c3-ab94-4207-a665-2709f2bf7e52',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '113',
   'date': 'Wed, 02 Mar 2022 04:02:34 GMT'},
  'RetryAttempts': 0}}

In [245]:
execution = pipeline_def.start()

#### To see the execution ID





In [222]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-southeast-1:120582440665:pipeline/awsworkshop-team1-pipeline-dev',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-southeast-1:120582440665:pipeline/awsworkshop-team1-pipeline-dev/execution/j37wof96jy72',
 'PipelineExecutionDisplayName': 'execution-1646117044228',
 'PipelineExecutionStatus': 'Executing',
 'PipelineExperimentConfig': {'ExperimentName': 'awsworkshop-team1-pipeline-dev',
  'TrialName': 'j37wof96jy72'},
 'CreationTime': datetime.datetime(2022, 3, 1, 6, 44, 3, 986000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 3, 1, 6, 44, 3, 986000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '567d6d2b-b104-4d20-ba37-348b50bcbc5f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '567d6d2b-b104-4d20-ba37-348b50bcbc5f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '541',
   'date': 'Tue, 01 Mar 2022 06:45:42 GMT'},
  'RetryAttempts': 0}}

In [223]:
execution.list_steps()

[{'StepName': 'awsworkshop-team1-inference-preprocessing',
  'StartTime': datetime.datetime(2022, 3, 1, 6, 44, 5, 92000, tzinfo=tzlocal()),
  'StepStatus': 'Executing',
  'AttemptCount': 0,
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-southeast-1:120582440665:processing-job/pipelines-j37wof96jy72-awsworkshop-team1-in-syeu22cdsn'}}},
 {'StepName': 'awsworkshop-team1-inference-createmodel',
  'StartTime': datetime.datetime(2022, 3, 1, 6, 44, 5, 92000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2022, 3, 1, 6, 44, 5, 717000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'AttemptCount': 0,
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:ap-southeast-1:120582440665:model/pipelines-j37wof96jy72-awsworkshop-team1-in-bybebxvp7g'}}}]

## Lineage
Review the lineage of the artifacts generated by the pipeline.

In [224]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer


viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    display(viz.show(pipeline_execution_step=execution_step))
    time.sleep(5)

{'StepName': 'awsworkshop-team1-inference-createmodel', 'StartTime': datetime.datetime(2022, 3, 1, 6, 44, 5, 92000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2022, 3, 1, 6, 44, 5, 717000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'AttemptCount': 0, 'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:ap-southeast-1:120582440665:model/pipelines-j37wof96jy72-awsworkshop-team1-in-bybebxvp7g'}}}


None

{'StepName': 'awsworkshop-team1-inference-preprocessing', 'StartTime': datetime.datetime(2022, 3, 1, 6, 44, 5, 92000, tzinfo=tzlocal()), 'StepStatus': 'Executing', 'AttemptCount': 0, 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-southeast-1:120582440665:processing-job/pipelines-j37wof96jy72-awsworkshop-team1-in-syeu22cdsn'}}}


Name/Source Direction     Type  \
0  s3://...2/input/code/preprocessing_config_inf.py     Input  DataSet   
1  s3://...nput_dataset/telco_cutomer_inference.csv     Input  DataSet   
2  12058...m/awsworkshop-preprocessing-image:latest     Input    Image   
3  s3://...wsworkshop/inference/2022-03-01/output1/    Output  DataSet   

  Association Type Lineage Type  
0    ContributedTo     artifact  
1    ContributedTo     artifact  
2    ContributedTo     artifact  
3         Produced     artifact